In [36]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import pickle

In [2]:
df = pd.read_csv('finfraud.csv')

In [11]:
# data = df.copy()

# #Ivde extracting nameOrig and nameDest where C is present
# data['OrigC']=data['nameOrig'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
# data['DestC']=data['nameDest'].apply(lambda x: 1 if str(x).find('C')==0 else 0)

# #Ivde creating new feature for transfer and cash_out
# data['TRANSFER']=data['type'].apply(lambda x: 1 if x=='TRANSFER' else 0)
# data['CASH_OUT']=data['type'].apply(lambda x: 1 if x=='CASH_OUT' else 0)

droplist=['isFlaggedFraud','type','nameDest','nameOrig']

#Preprocessing function 
def data_preprocessing(data):
    
    #Ivde extracting nameOrig and nameDest where C is present
    data['OrigC']=data['nameOrig'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
    data['DestC']=data['nameDest'].apply(lambda x: 1 if str(x).find('C')==0 else 0)

    #Ivde creating new feature for transfer and cash_out
    data['TRANSFER']=data['type'].apply(lambda x: 1 if x=='TRANSFER' else 0)
    data['CASH_OUT']=data['type'].apply(lambda x: 1 if x=='CASH_OUT' else 0)
    
    #Ivde extracting error in account balances
    data['Amount Error']=(abs(data['oldbalanceOrg']-data['newbalanceOrig'])-data['amount'])
       
    data = data.drop(labels=droplist,axis=1)
    
    return data

In [12]:
#Ivde extracting error in account balances
#data['Amount Error']=(abs(data['oldbalanceOrg']-data['newbalanceOrig'])-data['amount'])

In [13]:
#droplist=['isFlaggedFraud','type','nameDest','nameOrig']

In [14]:

def model_result(model,x_test,y_test):
    y_pred=model.predict(x_test)
    print('F1-score :',(f1_score(y_test,y_pred)))
    print('Confusion_matrix : ')
    print(confusion_matrix(y_test,y_pred))
    print("accuracy_score : ", end = '')
    print(accuracy_score(y_test,y_pred))
    print("classification_report")
    print(classification_report(y_test,y_pred))

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, classification_report, confusion_matrix

In [16]:
#train_df = data.drop(labels=droplist,axis=1)

df = data_preprocessing(df)

X = df.drop('isFraud',axis=1)
Y = df.isFraud

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,random_state=42, shuffle=False)

In [18]:
# %%time

# weights = (Y == 0).sum() / (1.0 * (Y == 1).sum()) # giving class weight
# clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
# clf.fit(X_train, y_train)
# print ('Test')
# model_result(clf,X_test,y_test)

In [19]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.4,random_state=42, shuffle=False)
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
model_result(xgb_model,X_test,y_test)

Test
AUPRC : 0.9999988164994575
F1 - score : 0.9993028582810476
Confusion_matrix : 
[[2540024       0]
 [      7    5017]]
accuracy_score
0.9999972495607156
classification_report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2540024
           1       1.00      1.00      1.00      5024

    accuracy                           1.00   2545048
   macro avg       1.00      1.00      1.00   2545048
weighted avg       1.00      1.00      1.00   2545048

Wall time: 5min 8s


In [20]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,random_state=42, shuffle=False)
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
model_result(rf_model,X_test,y_test)

Test
AUPRC : 0.9999997607600722
F1 - score : 0.9997811337272926
Confusion_matrix : 
[[1904216       0]
 [      2    4568]]
accuracy_score
0.9999989522136059
classification_report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1904216
           1       1.00      1.00      1.00      4570

    accuracy                           1.00   1908786
   macro avg       1.00      1.00      1.00   1908786
weighted avg       1.00      1.00      1.00   1908786

Wall time: 15min 13s


In [29]:
dataset = pd.read_csv('finfraud.csv')

In [30]:
dataset = dataset[dataset['isFraud']==1]

In [31]:
dataset = data_preprocessing(dataset)

In [32]:
X_data = dataset.drop('isFraud',axis=1)
y_data = dataset['isFraud']

In [33]:
model_result(xgb_model, X_data, y_data)

AUPRC : 1.0
F1 - score : 0.9975588917368485
Confusion_matrix : 
[[   0    0]
 [  40 8173]]
accuracy_score
0.9951296724704737
classification_report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00      8213

    accuracy                           1.00      8213
   macro avg       0.50      0.50      0.50      8213
weighted avg       1.00      1.00      1.00      8213



C:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
model_result(rf_model, X_data, y_data)

AUPRC : 1.0
F1 - score : 0.9998782269849001
Confusion_matrix : 
[[   0    0]
 [   2 8211]]
accuracy_score
0.9997564836235237
classification_report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00      8213

    accuracy                           1.00      8213
   macro avg       0.50      0.50      0.50      8213
weighted avg       1.00      1.00      1.00      8213



C:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
#XGBOOST MODEL
filename = 'xgboost.sav'
pickle.dump(xgb_model, open(filename, 'wb'))

In [39]:
#RANDOM FOREST MODEL
filename = 'randomforest.sav'
pickle.dump(rf_model, open(filename, 'wb'))

In [41]:
#PREPROCESSING FUNCTION
# filename = 'preprocessing.sav'
# pickle.dump(data_preprocessing, 'wb')